In [3]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install pip install python-slugify

In [4]:
import requests, random, logging, urllib.request, json
from bs4 import BeautifulSoup
from tqdm import tqdm
from slugify import slugify

In [5]:
logging.basicConfig(filename='app.log', filemode='w', format='%(name)s - %(levelname)s - %(message)s')

In [6]:
url = 'https://www.investopedia.com/financial-term-dictionary-4769738'

In [7]:
master_links = []

page = urllib.request.urlopen(url).read().decode('utf8','ignore') 
soup = BeautifulSoup(page,"lxml")

for link in soup.find_all('a',{'class': 'terms-bar__link mntl-text-link'},  href = True):

    master_links.append(link.get('href'))

In [8]:
master_links = master_links[0:27]

In [9]:
with open('URL_INDEX_BY_ALPHA.txt', 'w') as f:
    for item in master_links:
        f.write("%s\n" % item)

In [10]:
list_alpha = []

for articleIdx in master_links:
    
    page = urllib.request.urlopen(articleIdx).read().decode('utf8','ignore') 
    soup = BeautifulSoup(page,"lxml")
                         
    for link in soup.find_all('a',{'class': 'dictionary-top300-list__list mntl-text-link'},  href = True):
           
            list_alpha.append(link.get('href'))

In [11]:
with open('FULL_URL_INDEX.txt', 'w') as f:
    for item in list_alpha:
        f.write("%s\n" % item)

In [12]:
logf = open("error.log", "w")

In [13]:
# for article in tqdm(random.sample(list_alpha, 10)):
data = {} #json file
for article in tqdm(list_alpha):
    list_related = []
    body = []
    try:
        
        page = urllib.request.urlopen(article, timeout = 3).read().decode('utf8','ignore')
        soup = BeautifulSoup(page,"lxml")
    
        myTags = soup.find_all('p', {'class': 'comp mntl-sc-block finance-sc-block-html mntl-sc-block-html'})
    
        for link in soup.find_all('a',{'class': 'related-terms__title mntl-text-link'},  href = True):
            list_related.append(link.get('href'))
    
        title = slugify(soup.find('title').get_text(strip=True)) + '.json'
        data['name'] = soup.find('title').get_text(strip=True)
        data['@id'] = article
        data['related'] = list_related
        post = ''
    
        for tag in myTags:
            # body.append(str(tag.get_text(strip=True).encode('utf8', errors='replace'))) #get text content
            body.append(tag.decode_contents()) # get html content

        f = 'data/' + title
        data['body'] = body

        w = open(f, 'w')
        w.write(json.dumps(data))
        w.close()
        
    except:
        
        logf.write("Failed to extract: {0}\n".format(str(article)))
        logging.error("Exception occurred", exc_info=True)
        
    finally:
        
        pass

100%|██████████████████████████████████████████████████████████████████████████████| 6321/6321 [47:22<00:00,  2.22it/s]
